In [2]:
import numpy as np
import cv2
import heapq
import time
from math import dist

## Step 1: Define the action set

In [3]:
WHEEL_RADIUS = 33#/1000 # 33mm
ROBOT_RADIUS = 220#/1000 # 220mm
WHEEL_DISTANCE = 287#/1000 # 287mm
clearance = 20
clearance += ROBOT_RADIUS

rpm1, rpm2 = 50, 100
value = 20
turn = 7
# Declare the action cost set
# action_cost_set = [(0, rpm1, value), (rpm1, 0, value), (rpm1, rpm1, value), (0, rpm2, value),
#                    (rpm2, 0, value), (rpm2, rpm2, value), (rpm1, rpm2, value), (rpm2, rpm1, value)]

action_set = [(0, rpm1), (rpm1, 0), (rpm1, rpm1), (0, rpm2),
                   (rpm2, 0), (rpm2, rpm2), (rpm1, rpm2), (rpm2, rpm1)]

# action_cost_set = [(0, rpm1, value-turn), (rpm1, 0, value-turn), (rpm1, rpm1, value), (0, rpm2, value-turn),
#                    (rpm2, 0, value-turn), (rpm2, rpm2, value+turn), (rpm1, rpm2, value-1.5*turn), (rpm2, rpm1, value-1.5*turn)]

# action_cost_set = [(0, rpm1, 0+rpm1), (rpm1, 0, rpm1+0), (rpm1, rpm1, rpm1+rpm1), (0, rpm2, 0+rpm2),
#               (rpm2, 0, rpm2+0), (rpm2, rpm2, rpm2+rpm2), (rpm1, rpm2, rpm1+rpm2), (rpm2, rpm1, rpm2+rpm1)]

## Step 2: Mathematical representation of Free Space

In [4]:
width = 6000
height = 2000
scale = 5

clearance_color = (0, 255, 255)
obstacle_color = (0, 0, 0)
robot_radius_color = (255, 105, 180)

# Create a black canvas
canvas = np.zeros((height, width, 3), dtype="uint8")
# Create a white rectangle
canvas = cv2.rectangle(canvas, (clearance, clearance), (width-clearance, height-clearance), (255, 255, 255), -1)

# OBSTACLE 1
x1, x2 = 1500, 1750
y1, y2 = 0, 1000
# Draw the clearance
for i in range(x1-clearance, x2+clearance):
    for j in range(y1+clearance, y2+clearance):
        canvas[j, i] = clearance_color
# Draw the obstacle
for i in range(x1, x2):
    for j in range(y1, y2):
        canvas[j, i] = obstacle_color

# OBSTACLE 2
x1, x2 = 2500, 2750
y1, y2 = height-1000, height
# Draw the clearance
for i in range(x1-clearance, x2+clearance):
    for j in range(y1-clearance, y2-clearance+1):
        canvas[j, i] = clearance_color
# Draw the obstacle
for i in range(x1, x2):
    for j in range(y1, y2):
        canvas[j, i] = obstacle_color

# OBSTACLE 3
# Draw a circle at (4200, 800)
center = (4200, 800)
radius = 600
# Draw the clearance
canvas = cv2.circle(canvas, center, 600+clearance, clearance_color, -1)
# Draw the obstacle
canvas = cv2.circle(canvas, center, 600, obstacle_color, -1)
        
# x_center, y_center = 4200, 800
# radius = 600
# Draw the clearance
# for i in range(x_center-radius-clearance, x_center+radius+clearance):
#     for j in range(y_center-radius-clearance, y_center+radius+clearance):
#         if (i-x_center)**2 + (j-y_center)**2 <= (radius+clearance)**2 and canvas[j, i, 0] != 0:
#             canvas[j, i] = clearance_color
# # Draw the obstacle
# for i in range(x_center-radius, x_center+radius):
#     for j in range(y_center-radius, y_center+radius):
#         if (i-x_center)**2 + (j-y_center)**2 <= radius**2:
#             canvas[j, i] = obstacle_color

# x_start, y_start = clearance+1, clearance+1
# x_goal, y_goal = width-clearance-1, clearance+1

# Draw a circle at x_start, y_start
# canvas = cv2.circle(canvas, (x_start, y_start), 5, (0, 255, 0), -1)
# Draw a circle at x_goal, y_goal
# canvas = cv2.circle(canvas, (x_goal, y_goal), 5, (0, 0, 255), -1)


# Draw a red line at x = width/2
# canvas = cv2.line(canvas, (int(width/2.5), 0), (int(width/2.5), height), (0, 0, 255), 10)
# Resize the canvas by a factor of scale
width_resized = int(width/scale)
height_resized = int(height/scale)
canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
# cv2.imshow("Canvas", canvas_resized)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
# canvas

# canvas[y_goal, x_goal] 
# for i in range(height):
#     print(canvas[i, int(width/2.5)])

## Step 3: Implement A* search algorithm to search the tree and to find the optimal path

In [6]:
# import matplotlib.pyplot as plt
# import numpy as np
# import math
# def cost(Xi,Yi,Thetai,UL,UR):
#     t = 0
#     r = 0.038
#     L = 0.354
#     dt = 0.1
#     Xn=Xi
#     Yn=Yi
#     Thetan = 3.14 * Thetai / 180
#     # Xi, Yi,Thetai: Input point's coordinates
#     # Xs, Ys: Start point coordinates for plot function
#     # Xn, Yn, Thetan: End point coordintes
#     D=0
#     while t<1:
#         t = t + dt
#         Xs = Xn
#         Ys = Yn
#         Delta_Xn = 0.5*r * (UL + UR) * math.cos(Thetan) * dt
#         Delta_Yn = 0.5*r * (UL + UR) * math.sin(Thetan) * dt
#         Thetan += (r / L) * (UR - UL) * dt
#         D=D+ math.sqrt(math.pow((0.5*r * (UL + UR) * math.cos(Thetan) *
#         dt),2)+math.pow((0.5*r * (UL + UR) * math.sin(Thetan) * dt),2))
#         Thetan = 180 * (Thetan) / 3.14
#         return Xn, Yn, Thetan, D

# actions=[[5,5], [10,10],[5,0],[0,5],[5,10],[10,5]]
# for action in actions:
#     k = cost(0,0,45, action[0],action[1]) # (0,0,45) hypothetical start
# # configuration, this dosn't matter for calucating the edges'costs
#     print(k[3])

In [7]:
x, y, theta = 0, 0, 0
dt = 0.1

rpm1, rpm2 = 50, 100
value = 1

action_cost_set = [(0, rpm1, value), (rpm1, 0, value), (rpm1, rpm1, value), (0, rpm2, value),
                   (rpm2, 0, value), (rpm2, rpm2, value), (rpm1, rpm2, value), (rpm2, rpm1, value)]

distance_threshold = 0.5
angular_threshold = 30

# Make a lambda function to adjust the value of x to the visited space
adjust = lambda x, threshold: int(round(x*2)/2)/threshold 

for rpm_l, rpm_r, value in action_cost_set:
    # Convert the rpm values to angular velocity
    ul = 2 * np.pi * rpm_l / 60
    ur = 2 * np.pi * rpm_r / 60

    # Apply these velocities for 1 second to the model
    t = 0
    d = 0
    x_new, y_new, theta_new = x, y, theta
    while t < 0.4:
        dx_dt = WHEEL_RADIUS/2 * (ul + ur) * np.cos(np.radians(theta_new))
        dy_dt = WHEEL_RADIUS/2 * (ul + ur) * np.sin(np.radians(theta_new))
        dtheta_dt = np.rad2deg(WHEEL_RADIUS/WHEEL_DISTANCE * (ur - ul))

        # Get the new state
        x_new += dx_dt * dt
        y_new += dy_dt * dt
        theta_new += dtheta_dt * dt
        t += dt 
        # print("1")

        d += dist((x_new, y_new), (x, y))
    
    # Adjust each value
    x_adjusted = adjust(x_new, distance_threshold)
    y_adjusted = adjust(y_new, distance_threshold)
    theta_adjusted = adjust(theta_new, angular_threshold)

    print(f"x_new: {np.round(x_new, 2)}, y_new: {np.round(y_new, 2)}, theta_new: {np.round(theta_new, 2)}, ul: {np.round(ul, 2)}, ur: {np.round(ur, 2)}")
    print(f"x_adjusted: {x_adjusted}, y_adjusted: {y_adjusted}, theta_adjusted: {theta_adjusted}")
    print(f"Distance: {d/4}")
    print('\n')


x_new: 34.34, y_new: 3.11, theta_new: 13.8, ul: 0.0, ur: 5.24
x_adjusted: 68.0, y_adjusted: 6.0, theta_adjusted: 0.4666666666666667
Distance: 21.569106884398675


x_new: 34.34, y_new: -3.11, theta_new: -13.8, ul: 5.24, ur: 0.0
x_adjusted: 68.0, y_adjusted: -6.0, theta_adjusted: -0.4666666666666667
Distance: 21.569106884398675


x_new: 69.12, y_new: 0.0, theta_new: 0.0, ul: 5.24, ur: 5.24
x_adjusted: 138.0, y_adjusted: 0.0, theta_adjusted: 0.0
Distance: 43.196898986859665


x_new: 67.38, y_new: 12.3, theta_new: 27.6, ul: 0.0, ur: 10.47
x_adjusted: 134.0, y_adjusted: 24.0, theta_adjusted: 0.9
Distance: 42.96250883719149


x_new: 67.38, y_new: -12.3, theta_new: -27.6, ul: 10.47, ur: 0.0
x_adjusted: 134.0, y_adjusted: -24.0, theta_adjusted: -0.9
Distance: 42.96250883719149


x_new: 138.23, y_new: 0.0, theta_new: 0.0, ul: 10.47, ur: 10.47
x_adjusted: 276.0, y_adjusted: 0.0, theta_adjusted: 0.0
Distance: 86.39379797371933


x_new: 103.02, y_new: 9.33, theta_new: 13.8, ul: 5.24, ur: 10.47
x_a

In [12]:
x_start, y_start, theta_start = clearance+1, clearance+1, 0
x_goal, y_goal = width-clearance-1, clearance+1
# x_goal, y_goal = width-clearance-1, height-clearance+1

# Draw a circle at x_start, y_start
# canvas = cv2.circle(canvas, (x_start, y_start), 50, (0, 255, 0), -1)
# Draw a circle at x_goal, y_goal
# canvas = cv2.circle(canvas, (x_goal, y_goal), 50, (0, 0, 255), -1)

distance_threshold = 20
angular_threshold = 30
dt = 0.1

# Make a lambda function to adjust the value of x to the visited space
adjust = lambda x, threshold: int(int(round(x*2)/2)/threshold)

q = []
heapq.heappush(q, (0, x_start, y_start, theta_start))

# Dictionary to store visited nodes
visited = {(adjust(x_start, distance_threshold),
            adjust(y_start, distance_threshold),
            adjust(theta_start, angular_threshold)): 1}

# Dictionary to store the parent of each node
parent = {(x_start, y_start, theta_start): (x_start, y_start, theta_start)}

# Dictionary to store the cost to come of each node
cost_to_come = {(adjust(x_start, distance_threshold),
                 adjust(y_start, distance_threshold),
                 adjust(theta_start, angular_threshold)): 0}

# Dictionary to store the cost of each node
cost = {(adjust(x_start, distance_threshold),
         adjust(y_start, distance_threshold),
         adjust(theta_start, angular_threshold)): 0}

# Dictionary to store the inputs applied to that node
inputs = {(x_start, y_start, theta_start): (0, 0)}

reached = False
start = time.time()

while q:
    
    _, x, y, theta = heapq.heappop(q)
    x_achieved, y_achieved, theta_achieved = x, y, theta

    # Get the cost to come of the current node
    c2c = cost_to_come[(adjust(x, distance_threshold),
                        adjust(y, distance_threshold),
                        adjust(theta, angular_threshold))]

    if dist((x, y), (x_goal, y_goal)) < 10:
        end = time.time()
        print("Goal reached")
        # Print time in minutes and seconds
        print("Time taken: ", int((end-start)/60), "minutes", int((end-start)%60), "seconds")
        # print("Goal reached: ", end-start, "seconds")
        reached = True
        x_achieved, y_achieved, theta_achieved = x, y, theta
        break

    for rpm_l, rpm_r in action_set:

        # Convert the rpm values to angular velocity
        ul = 2 * np.pi * rpm_l / 60
        ur = 2 * np.pi * rpm_r / 60

        # Apply these velocities for T seconds to the model
        t = 0
        dt = 0.1
        d = 0
        T = 0.3
        x_new, y_new, theta_new = x, y, theta
        while t < T:
            dx_dt = WHEEL_RADIUS/2 * (ul + ur) * np.cos(np.radians(theta_new))
            dy_dt = WHEEL_RADIUS/2 * (ul + ur) * np.sin(np.radians(theta_new))
            dtheta_dt = np.rad2deg(WHEEL_RADIUS/WHEEL_DISTANCE * (ur - ul))
            
            # Save the current state
            x_prev, y_prev, theta_prev = x_new, y_new, theta_new

            # Get the new state
            x_new += dx_dt * dt
            y_new += dy_dt * dt
            theta_new += dtheta_dt * dt 

            if canvas[int(round(y_new*2)/2), int(round(x_new*2)/2), 0] == 255:
                # Calculate the total distance travelled
                d += np.sqrt( (dx_dt*dt)**2 + (dy_dt*dt)**2)
                t += dt 
            else:
                x_new, y_new, theta_new = x_prev, y_prev, theta_prev
                break

        # d = dist((x_new, y_new), (x, y))
        # Let the action cost be a function of distance travelled
        action_cost = int(d)

        # print("x_new: ", x_new, "y_new: ", y_new, "theta_new: ", theta_new, "ul: ", ul, "ur: ", ur)

        # Keep the theta_newing angle within 180 and -180
        if theta_new > 180:
            theta_new -= 360
        elif theta_new < -180:
            theta_new += 360

        # Cap the new node values within the boundaries of the canvas
        x_new = max(clearance, min(width-clearance, x_new))
        y_new = max(clearance, min(height-clearance, y_new))

        # Adjust the values for the canvas
        x_cvs = int(round(x_new*2)/2)
        y_cvs = int(round(y_new*2)/2)
        theta_cvs = int(round(theta_new*2)/2)

        # Adjust the values for the visited dictionary
        x_vis = adjust(x_new, distance_threshold)
        y_vis = adjust(y_new, distance_threshold)
        theta_vis = adjust(theta_cvs, angular_threshold)

        # Check if the new node is within the boundaries of the canvas
        if 0 <= x_new < width and 0 <= y_new < height and canvas[y_cvs, x_cvs, 0] == 255:

            # Check if the new node is not visited
            if (x_vis, y_vis, theta_vis) not in visited:
                # Store the parent of the node
                parent[(x_new, y_new, theta_new)] = (x, y, theta)
                # Store the cost to come of the node
                cost_to_come[(x_vis, y_vis, theta_vis)] = c2c + action_cost
                # Store the cost of the node
                cost[(x_vis, y_vis, theta_vis)] = cost_to_come[(x_vis, y_vis, theta_vis)] + dist((x_new, y_new), (x_goal, y_goal))
                # Push the node into the priority queue
                heapq.heappush(q, (cost[(x_vis, y_vis, theta_vis)], x_new, y_new, theta_new))
                # Mark the node as visited
                visited[(x_vis, y_vis, theta_vis)] = 1
                # Store the inputs applied to that node
                inputs[(x_new, y_new, theta_new)] = (rpm_l, rpm_r)

            # If the node is visited, check if the new cost is less than the previous cost
            elif cost_to_come[(x_vis, y_vis, theta_vis)] > c2c + action_cost: 
                parent[(x_new, y_new, theta_new)] = (x, y, theta)
                cost_to_come[(x_vis, y_vis, theta_vis)] = c2c + action_cost 
                cost[x_vis, y_vis, theta_vis] = cost_to_come[(x_vis, y_vis, theta_vis)] + dist((x_new, y_new), (x_goal, y_goal))
                # Store the inputs applied to that node
                inputs[(x_new, y_new, theta_new)] = (rpm_l, rpm_r)

if not reached:
    print('Goal could not be reached')
    print("Exiting...")

Goal reached
Time taken:  0 minutes 25 seconds


## Step 4: Optimal Path

In [30]:
# Get the path from the parent dictionary
path = []
# x, y = x_goal, y_goal   
x, y, theta = x_achieved, y_achieved, theta_achieved
while (x, y, theta) != (x_start, y_start, theta_start):
    # print(x, y)
    rpm_l, rpm_r = inputs[(x, y, theta)]
    path.append((x, y, theta, rpm_l, rpm_r))
    x, y, theta = parent[(x, y, theta)]
rpm_l, rpm_r = inputs[(x, y, theta)]
path.append((x, y, theta, rpm_l, rpm_r))
path.reverse()

In [20]:
path

[(241, 241, 0, 0),
 (292.8362787842316, 241.0, 50, 50),
 (370.35608014862635, 245.67270621840194, 50, 100),
 (445.7755330581259, 264.19458273808783, 50, 100),
 (494.2664733915471, 282.51474932513514, 50, 50),
 (539.9839738821689, 306.4110330835863, 0, 100),
 (578.8706925649041, 340.68672284012365, 50, 50),
 (656.6441299303743, 409.23810235319837, 100, 100),
 (695.5308486131095, 443.51379210973573, 50, 50),
 (734.4175672958446, 477.7894818662731, 50, 50),
 (789.4819072963004, 532.5532677771552, 50, 100),
 (804.2589772628459, 553.8080649240819, 0, 50),
 (836.4148338121706, 624.498620704596, 50, 100),
 (867.6949799396217, 723.339663045476, 100, 100),
 (889.0545394717044, 770.2959346731279, 100, 0),
 (953.2487320112641, 851.7029665375819, 100, 100),
 (1017.4429245508238, 933.1099984020359, 100, 100),
 (1081.6371170903835, 1014.5170302664899, 100, 100),
 (1113.734213360163, 1055.2205461987169, 50, 50),
 (1158.0654232597994, 1118.9849376394973, 50, 100),
 (1198.4797159853308, 1185.3011402635

## Step 5: Represent the Optimal Path

In [15]:
def obstacles():
    width = 6000
    height = 2000
    scale = 5

    clearance_color = (0, 255, 255)
    obstacle_color = (0, 0, 0)

    # Create a black canvas
    canvas = np.zeros((height, width, 3), dtype="uint8")
    # Create a white rectangle
    canvas = cv2.rectangle(canvas, (clearance, clearance), (width-clearance, height-clearance), (255, 255, 255), -1)

    # OBSTACLE 1
    x1, x2 = 1500, 1750
    y1, y2 = 0, 1000
    # Draw the clearance
    for i in range(x1-clearance, x2+clearance):
        for j in range(y1+clearance, y2+clearance):
            canvas[j, i] = clearance_color
    # Draw the obstacle
    for i in range(x1, x2):
        for j in range(y1, y2):
            canvas[j, i] = obstacle_color

    # OBSTACLE 2
    x1, x2 = 2500, 2750
    y1, y2 = height-1000, height
    # Draw the clearance
    for i in range(x1-clearance, x2+clearance):
        for j in range(y1-clearance, y2-clearance+1):
            canvas[j, i] = clearance_color
    # Draw the obstacle
    for i in range(x1, x2):
        for j in range(y1, y2):
            canvas[j, i] = obstacle_color

    # OBSTACLE 3
    # Draw a circle at (4200, 800)
    center = (4200, 800)
    radius = 600
    # Draw the clearance
    canvas = cv2.circle(canvas, center, 600+clearance, clearance_color, -1)
    # Draw the obstacle
    canvas = cv2.circle(canvas, center, 600, obstacle_color, -1)
            
    # x_center, y_center = 4200, 800
    # radius = 600
    # Draw the clearance
    # for i in range(x_center-radius-clearance, x_center+radius+clearance):
    #     for j in range(y_center-radius-clearance, y_center+radius+clearance):
    #         if (i-x_center)**2 + (j-y_center)**2 <= (radius+clearance)**2 and canvas[j, i, 0] != 0:
    #             canvas[j, i] = clearance_color
    # # Draw the obstacle
    # for i in range(x_center-radius, x_center+radius):
    #     for j in range(y_center-radius, y_center+radius):
    #         if (i-x_center)**2 + (j-y_center)**2 <= radius**2:
    #             canvas[j, i] = obstacle_color

    # x_start, y_start = clearance+1, clearance+1
    # x_goal, y_goal = width-clearance-1, clearance+1

    # Draw a circle at x_start, y_start
    # canvas = cv2.circle(canvas, (x_start, y_start), 5, (0, 255, 0), -1)
    # Draw a circle at x_goal, y_goal
    # canvas = cv2.circle(canvas, (x_goal, y_goal), 5, (0, 0, 255), -1)


    # Draw a red line at x = width/2
    # canvas = cv2.line(canvas, (int(width/2.5), 0), (int(width/2.5), height), (0, 0, 255), 10)
    # Resize the canvas by a factor of scale
    width_resized = int(width/scale)
    height_resized = int(height/scale)
    canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
    # cv2.imshow("Canvas", canvas_resized)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return canvas

In [36]:
canvas = obstacles()

# Start a video writer in mp4 format
astar1 = cv2.VideoWriter('astar1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 50, (width_resized, height_resized))

# Draw the start and goal nodes on the canvas
cv2.circle(canvas, (x_start, y_start), 10, (0, 255, 0), -1)
cv2.circle(canvas, (x_goal, y_goal), 10, (0, 165, 255), -1)

# Draw on every threshold frame
threshold = 500
counter = 0

# # Draw the visited nodes on the canvas as an arrow going from the parent to the child
# for x, y, theta in parent:
#     counter += 1
#     cv2.arrowedLine(canvas, (int(parent[(x, y, theta)][0]), int(parent[(x, y, theta)][1])), (int(x), int(y)), (255, 0, 0), 5)
#     # cv2.circle(canvas, (int(x), int(y)), 1, (255, 0, 0), 10)
#     if(counter == threshold):
#         # Resize the canvas by a factor of scale
#         canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
#         cv2.imshow('Canvas', canvas_resized)
#         # cv2.imshow('Canvas', canvas)
#         astar.write(canvas_resized)
#         cv2.waitKey(1)  
#         counter = 0

# Draw the visited nodes on the canvas as a curve going from the parent to the child
for x, y, theta in parent:
    counter += 1
    # Plot this point on the canvas
    # cv2.circle(canvas, (int(x), int(y)), 1, (255, 0, 0), 10)
    # Plot the curve from the parent to the child
    for rpm_l, rpm_r in action_set:
        ul = 2 * np.pi * rpm_l / 60
        ur = 2 * np.pi * rpm_r / 60
        # Apply these velocities for t seconds to the model
        t = 0
        dt = 0.1
        x_new, y_new, theta_new = x, y, theta
        x, y = x_new, y_new
        while t < T:
            dx_dt = WHEEL_RADIUS/2 * (ul + ur) * np.cos(np.radians(theta_new))
            dy_dt = WHEEL_RADIUS/2 * (ul + ur) * np.sin(np.radians(theta_new))
            dtheta_dt = np.rad2deg(WHEEL_RADIUS/WHEEL_DISTANCE * (ur - ul))
            
            # Get the new state
            x_new += dx_dt * dt
            y_new += dy_dt * dt
            theta_new += dtheta_dt * dt 
            # Plot this point on the canvas
            x_cvs = int(round(x_new*2)/2)
            y_cvs = int(round(y_new*2)/2)
            # x_cvs = int(x_new)
            # y_cvs = int(y_new)
            if clearance <= x_new < width-clearance-1 and clearance <= y_new < height-clearance-1 and canvas[y_cvs, x_cvs, 0] == 255:
            # if canvas[y_cvs, x_cvs, 0] == 255:
                cv2.line(canvas, (int(x), int(y)), (int(x_cvs), int(y_cvs)), (254, 0, 0), 5)
                # cv2.circle(canvas, (int(x_cvs), int(y_cvs)), 1, (255, 0, 0), 10)
                x, y = x_cvs, y_cvs
                t += dt 
            else:
                break
    if(counter == threshold):
        # Resize the canvas by a factor of scale
        canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
        cv2.imshow('Canvas', canvas_resized)
        # cv2.imshow('Canvas', canvas)
        astar1.write(canvas_resized)
        cv2.waitKey(1)  
        counter = 0

    
# Draw the start and goal nodes on the canvas
cv2.circle(canvas, (x_start, y_start), 10, (0, 255, 0), -1)
cv2.circle(canvas, (x_goal, y_goal), 10, (0, 165, 255), -1)

# Draw the path on the canvas
for i in range(len(path)-1):
    # Draw a red dot at path points
    # cv2.circle(canvas, (int(path[i][0]), int(path[i][1])), 1, (0, 0, 255), 10)
    # Draw a line connecting the path points
    cv2.line(canvas, (int(path[i][0]), int(path[i][1])), (int(path[i+1][0]), int(path[i+1][1])), (0, 0, 255), 10)
    # Resize the canvas by a factor of scale
    canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
    cv2.imshow('Canvas', canvas_resized)
    # cv2.imshow('Canvas', canvas)
    astar1.write(canvas_resized)

x, y, theta = path[0][0], path[0][1], path[0][2]

for _, _, _, rpm_l, rpm_r in path:
    # Convert the rpm values to angular velocity
    ul = 2 * np.pi * rpm_l / 60
    ur = 2 * np.pi * rpm_r / 60

    # Apply these velocities for T seconds to the model
    t = 0
    dt = 0.1
    x_parent, y_parent, theta_parent = x, y, theta
    while t < T:
        dx_dt = WHEEL_RADIUS/2 * (ul + ur) * np.cos(np.radians(theta))
        dy_dt = WHEEL_RADIUS/2 * (ul + ur) * np.sin(np.radians(theta))
        dtheta_dt = np.rad2deg(WHEEL_RADIUS/WHEEL_DISTANCE * (ur - ul))
        
        # Get the new state
        x += dx_dt * dt
        y += dy_dt * dt
        theta += dtheta_dt * dt 
        t += dt  
    
    cv2.line(canvas, (int(x_parent), int(y_parent)), (int(x), int(y)), robot_radius_color, 10)
            
        
    # Resize the canvas by a factor of scale
    canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
    cv2.imshow('Canvas', canvas_resized)
    # cv2.imshow('Canvas', canvas)
    astar1.write(canvas_resized)
    cv2.waitKey(1)
    

# Release VideoWriter
astar1.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
def obstacles():
    width = 6000
    height = 2000
    scale = 5

    clearance_color = (0, 255, 255)
    obstacle_color = (0, 0, 0)

    # Create a black canvas
    canvas = np.zeros((height, width, 3), dtype="uint8")
    # Create a white rectangle
    canvas = cv2.rectangle(canvas, (clearance, clearance), (width-clearance, height-clearance), (255, 255, 255), -1)

    # OBSTACLE 1
    x1, x2 = 1500, 1750
    y1, y2 = 0, 1000
    # Draw the clearance
    for i in range(x1-clearance, x2+clearance):
        for j in range(y1+clearance, y2+clearance):
            canvas[j, i] = clearance_color
    # Draw the obstacle
    for i in range(x1, x2):
        for j in range(y1, y2):
            canvas[j, i] = obstacle_color

    # OBSTACLE 2
    x1, x2 = 2500, 2750
    y1, y2 = height-1000, height
    # Draw the clearance
    for i in range(x1-clearance, x2+clearance):
        for j in range(y1-clearance, y2-clearance+1):
            canvas[j, i] = clearance_color
    # Draw the obstacle
    for i in range(x1, x2):
        for j in range(y1, y2):
            canvas[j, i] = obstacle_color

    # OBSTACLE 3
    # Draw a circle at (4200, 800)
    center = (4200, 800)
    radius = 600
    # Draw the clearance
    canvas = cv2.circle(canvas, center, 600+clearance, clearance_color, -1)
    # Draw the obstacle
    canvas = cv2.circle(canvas, center, 600, obstacle_color, -1)
            
    # x_center, y_center = 4200, 800
    # radius = 600
    # Draw the clearance
    # for i in range(x_center-radius-clearance, x_center+radius+clearance):
    #     for j in range(y_center-radius-clearance, y_center+radius+clearance):
    #         if (i-x_center)**2 + (j-y_center)**2 <= (radius+clearance)**2 and canvas[j, i, 0] != 0:
    #             canvas[j, i] = clearance_color
    # # Draw the obstacle
    # for i in range(x_center-radius, x_center+radius):
    #     for j in range(y_center-radius, y_center+radius):
    #         if (i-x_center)**2 + (j-y_center)**2 <= radius**2:
    #             canvas[j, i] = obstacle_color

    # x_start, y_start = clearance+1, clearance+1
    # x_goal, y_goal = width-clearance-1, clearance+1

    # Draw a circle at x_start, y_start
    # canvas = cv2.circle(canvas, (x_start, y_start), 5, (0, 255, 0), -1)
    # Draw a circle at x_goal, y_goal
    # canvas = cv2.circle(canvas, (x_goal, y_goal), 5, (0, 0, 255), -1)


    # Draw a red line at x = width/2
    # canvas = cv2.line(canvas, (int(width/2.5), 0), (int(width/2.5), height), (0, 0, 255), 10)
    # Resize the canvas by a factor of scale
    width_resized = int(width/scale)
    height_resized = int(height/scale)
    canvas_resized = cv2.resize(canvas, (width_resized, height_resized))
    # cv2.imshow("Canvas", canvas_resized)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return canvas

In [25]:
# import matplotlib.pyplot as plt
# import numpy as np
# import math
# fig, ax = plt.subplots()
# def plot_curve(Xi,Yi,Thetai,UL,UR):
#     t = 0
#     r = 0.038
#     L = 0.354
#     dt = 0.1
#     Xn=Xi
#     Yn=Yi
#     Thetan = 3.14 * Thetai / 180
#     # Xi, Yi,Thetai: Input point's coordinates
#     # Xs, Ys: Start point coordinates for plot function
#     # Xn, Yn, Thetan: End point coordintes
#     D=0
#     while t<1:
#         t = t + dt
#         Xs = Xn
#         Ys = Yn
#         Xn += 0.5*r * (UL + UR) * math.cos(Thetan) * dt
#         Yn += 0.5*r * (UL + UR) * math.sin(Thetan) * dt
#         Thetan += (r / L) * (UR - UL) * dt
#         plt.plot([Xs, Xn], [Ys, Yn], color="blue")
#         Thetan = 180 * (Thetan) / 3.14
#     return Xn, Yn, Thetan, D

# actions=[[5,5], [10,10],[5,0],[0,5],[5,10],[10,5]]
# for action in actions:
#     X1= plot_curve(0,0,45, action[0],action[1]) # (0,0,45) hypothetical start configuration
# for action in actions:
#     X2=plot_curve(X1[0],X1[1],X1[2], action[0],action[1])
# plt.grid()
# ax.set_aspect('equal')
# plt.xlim(0,1)
# plt.ylim(0,1)
# plt.title('How to plot a vector in matplotlib ?',fontsize=10)
# plt.show()
# plt.close()
